In [2]:
import mat_properties as prop

# Считывание рефпропа
RP = prop.init_REFPROP(r"C:\Program Files (x86)\REFPROP")
gasmix = "Nitrogen*Oxygen*CO2*Water*Argon"
fractiongas = (
    0.715784461314205,
    0.0114726415414296,
    0.143332198252282,
    0.116991486761544,
    0.0124192121305392
)


water = prop.Materials_prop(
    "Water",
    [1.0, 0, 0, 0, 0],
    prop.REFPROP_h_s,
    prop.REFPROP_p_t,
    prop.REFPROP_p_h,
    prop.REFPROP_p_s,
    prop.REFPROP_p_q,
    prop.REFPROP_t_q,
    prop.REFPROP_p_rho,
    prop.REFPROP_s_q,
    RP=RP,
)


Methane = prop.Materials_prop(
    "METHANE",
    [1.0, 0, 0, 0, 0],
    prop.REFPROP_h_s,
    prop.REFPROP_p_t,
    prop.REFPROP_p_h,
    prop.REFPROP_p_s,
    prop.REFPROP_p_q,
    prop.REFPROP_t_q,
    prop.REFPROP_p_rho,
    prop.REFPROP_s_q,
    RP=RP,
)


fractionwaterMethane = (0.848859488304273, 0.151140511695727, 0, 0, 0)
waterMethanemix = "Water*METHANE"

waterMethane = prop.Materials_prop(
    waterMethanemix,
    fractionwaterMethane,
    prop.REFPROP_h_s,
    prop.REFPROP_p_t,
    prop.REFPROP_p_h,
    prop.REFPROP_p_s,
    prop.REFPROP_p_q,
    prop.REFPROP_t_q,
    prop.REFPROP_p_rho,
    prop.REFPROP_s_q,
    RP=RP,
)

gas = prop.Materials_prop(
    gasmix,
    fractiongas,
    prop.REFPROP_h_s,
    prop.REFPROP_p_t,
    prop.REFPROP_p_h,
    prop.REFPROP_p_s,
    prop.REFPROP_p_q,
    prop.REFPROP_t_q,
    prop.REFPROP_p_rho,
    prop.REFPROP_s_q,
    RP=RP,
)


G1 = 47.56
T11 = 509
P11 = 7.845


def calculate_steam_transformer(dT=15, mindT=5):
    H11 = water.p_t(P11, T11)['h']
    Q11 = water.p_h(P11, H11)['Q']

    Q12 = 0  # конденсация греющего пара
    P12 = P11
    H12 = water.p_q(P12, Q12)['h']
    T12 = water.p_q(P12, Q12)['T']

    P13 = 2
    T13 = water.p_h(P13, H12)['T']
    Q13 = water.p_h(P13, H12)['Q']
    H13 = H12
    
    P22 = 2
    T22 = T11-dT
    H22 = water.p_t(P22, T22)['h']
    Q22 = water.p_h(P22, H22)['Q']

    T21 = T13-mindT
    P21 = P22
    H21 = water.p_t(P21, T21)['h']
    Q21 = water.p_h(P21, H21)['Q']

    G2 = G1 * ((H11-H12)/(H22-H21))
    return G2


G2 = calculate_steam_transformer()


def calculate_reformer(G2, Trefromer_out=700, Pressure_reform=2):

    Gmethane = (0.151140511695727/0.848859488304273)*G2
    Greformer = Gmethane + G2

    # Или не Pr (2000), а 2500 как в аспене?
    Tmethane = 121.53615158005 #выход с компрессора
    Hmet = Methane.p_t(Pressure_reform, Tmethane)['h']
    H1r = (G2*H22 + Gmet*Hmet) / Greformer
    H2r = waterMethane.p_t(Pressure_reform, Trefromer_out)['h']
    Qdt = Greformer*(H2r-H1r)/1000
    Qr = (52166.505091208484/44.6397313913235)*Greformer/1000
    Qref = Qdt+Qr
    T1gas = 1969
    T2gas = 800
    H1gas = gas.p_t(0.1, T1gas)['h']
    H2gas = gas.p_t(0.1, T2gas)['h']
    Ggas = Qref/(H1gas-H2gas)
    Gair = Ggas*48.5966429877363/51.2672005145991
    Gch4 = Ggas*2.67055752686283/51.2672005145991
    res_out={'Greformer':Greformer,'Ggas':Ggas,'Gair':Gair,'Gch4':Gch4,'Qref':Qref,'Qdt':Qdt,'Qr':Qr}
    return res_out

Reform_out=calculate_reformer(G2)
print(Reform_out)

0.21237259075516854


NameError: name 'H22' is not defined